In [1]:
include("../../src/JWAS.jl")

JWAS

In [2]:
using DataFrames
using JWAS: MT,Datasets

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

In [4]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [5]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [6]:
model_equations = "BW = intercept + age + sex + Animal"
 #                  CW = intercept + age + sex + Animal";

In [8]:
model    = MT.build_model(model_equations,R[1,1]);

LoadError: LoadError: MethodError: `build_model` has no method matching build_model(::ASCIIString, ::Float64)
Closest candidates are:
  build_model(::AbstractString, !Matched::Array{Float64,2})
while loading In[8], in expression starting on line 1

In [9]:
MT.set_covariate(model,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [10]:
ped = MT.get_pedigree(pedfile);

Finished!


In [11]:
MT.set_random(model,"Animal", ped,G)

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [12]:
out2 = MT.runMCMC(model,data,chain_length=1000,printout_frequency=500,output_samples_frequency=10);

LoadError: LoadError: MethodError: `values` has no method matching values(::Float64)
while loading In[12], in expression starting on line 1

In [15]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Posterior mean of residual covariance matrix"
  "MCMC samples for residual covariance matrix"
  "MCMC samples for polygenic effects covariance matrix"
  "Posterior mean of location parameters"